<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 10


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----
Создать базовый класс Invoice в C#, который будет представлять информацию о
фактурах за поставленные товары или оказанные услуги. На основе этого класса
разработать 2-3 производных класса, демонстрирующих принципы наследования и
полиморфизма. В каждом из классов должны быть реализованы новые атрибуты и
методы, а также переопределены некоторые методы базового класса для
демонстрации полиморфизма.

Требования к базовому классу Invoice:

• Атрибуты: 
•Номер фактуры (InvoiceNumber)
•Дата выдачи (IssueDate)
•Общая сумма (TotalAmount).

• Методы:

o CalculateTotal(): метод для расчета общей суммы по фактуре.
o AddLine(LineItem lineItem): метод для добавления позиции в фактуру.
o RemoveLine(LineItem lineItem): метод для удаления позиции из
фактуры.

Требования к производным классам:
1. ТоварнаяФактура (GoodsInvoice): Должна содержать дополнительные
атрибуты, такие как Дата поставки (SupplyDate). Метод AddLine() должен
быть переопределен для добавления информации о дате поставки товара
при добавлении позиции.
2. УслуговаяФактура (ServiceInvoice): Должна содержать дополнительные
атрибуты, такие как Дата оказания услуги (ServiceDate).
Метод RemoveLine() должен быть переопределен для добавления
информации о причине аннулирования услуги при удалении позиции.
3. КомбинированнаяФактура (CombinedInvoice) (если требуется третий класс):
Должна содержать дополнительные атрибуты, такие как Наличие возврата
(ReturnAllowed). Метод CalculateTotal() должен быть переопределен для
учета возможного возврата товара или услуги при расчете общей суммы.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) создайте явную реализации интерфейса и управление зависимостями 


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [ ]:
using System;
using System.Collections.Generic;

public interface ILineItem
{
    string Description { get; set; }
    decimal Price { get; set; }
    int Quantity { get; set; }
    decimal Discount { get; set; }
    decimal GetTotal();
    void CalculateTotal();
}

public interface IPrintable
{
    void PrintInvoice();
}

public interface ICalculable
{
    void CalculateTotal();
}

public abstract class BaseLineItem : ILineItem, ICalculable
{
    public string Description { get; set; }
    public decimal Price { get; set; }
    public int Quantity { get; set; }
    public decimal Discount { get; set; }

    protected BaseLineItem(string description, decimal price, int quantity, decimal discount = 0)
    {
        Description = description;
        Price = price;
        Quantity = quantity;
        Discount = discount;
    }

    public abstract decimal GetTotal();

    public virtual void CalculateTotal()
    {
        Console.WriteLine($"Расчёт общей суммы для: {Description}, Количество: {Quantity}, Цена: {Price}, Скидка: {Discount}");
    }
}

public class LineItem : BaseLineItem
{
    public string SKU { get; set; }

    public LineItem(string description, decimal price, int quantity, decimal discount = 0, string sku = null)
        : base(description, price, quantity, discount)
    {
        SKU = sku;
    }

    public override decimal GetTotal()
    {
        return Price * Quantity - Discount;
    }
}

public class ServiceLineItem : BaseLineItem
{
    public decimal ServiceFee { get; set; }
    public string ServiceType { get; set; }

    public ServiceLineItem(string description, decimal price, int quantity, decimal serviceFee, decimal discount = 0, string serviceType = "")
        : base(description, price, quantity, discount)
    {
        ServiceFee = serviceFee;
        ServiceType = serviceType;
    }

    public override decimal GetTotal()
    {
        return (Price + ServiceFee) * Quantity - Discount;
    }

    public override void CalculateTotal()
    {
        base.CalculateTotal();
        Console.WriteLine($"Итог для услуги: {GetTotal()}");
    }
}

public class ItemCollection<T> where T : ILineItem
{
    private List<T> items = new List<T>();

    public void AddItem(T item)
    {
        items.Add(item);
    }

    public void RemoveItem(T item)
    {
        items.Remove(item);
    }

    public T FindItemByDescription(string description)
    {
        foreach (var item in items)
        {
            if (item.Description == description)
                return item;
        }
        return default(T);
    }

    public void DisplayAllItems()
    {
        foreach (var item in items)
        {
            Console.WriteLine($"Товар/Услуга: {item.Description}, Цена: {item.Price}, Количество: {item.Quantity}, Скидка: {item.Discount}, Общая сумма: {item.GetTotal()}");
        }
    }

    public void CalculateTotal()
    {
        decimal total = 0;
        foreach (var item in items)
        {
            total += item.GetTotal();
        }
        Console.WriteLine($"Общая сумма фактуры: {total}");
    }
}

public abstract class Invoice : IPrintable, ICalculable
{
    public string InvoiceNumber { get; set; }
    public DateTime IssueDate { get; set; }
    public decimal TotalAmount { get; set; }
    public string CustomerName { get; set; }
    public string PaymentMethod { get; set; }
    public string Currency { get; set; }

    protected List<ILineItem> LineItems;

    public Invoice(string invoiceNumber, DateTime issueDate, string customerName, string paymentMethod, string currency, List<ILineItem> lineItems)
    {
        InvoiceNumber = invoiceNumber;
        IssueDate = issueDate;
        CustomerName = customerName;
        PaymentMethod = paymentMethod;
        Currency = currency;
        LineItems = lineItems ?? new List<ILineItem>();
        TotalAmount = 0;
    }

    void ICalculable.CalculateTotal()
    {
        CalculateTotal();
    }

    void IPrintable.PrintInvoice()
    {
        PrintInvoice();
    }

    public virtual void AddLine(ILineItem lineItem)
    {
        LineItems.Add(lineItem);
        CalculateTotal();
    }

    public virtual void RemoveLine(ILineItem lineItem)
    {
        LineItems.Remove(lineItem);
        CalculateTotal();
    }

    public virtual void CalculateTotal()
    {
        TotalAmount = 0;
        foreach (var item in LineItems)
        {
            TotalAmount += item.GetTotal();
        }
        Console.WriteLine($"Общая сумма счета: {TotalAmount}");
    }

    // Метод для печати счета
    public virtual void PrintInvoice()
    {
        Console.WriteLine($"Номер фактуры: {InvoiceNumber}, Дата выпуска: {IssueDate.ToShortDateString()}, " +
            $"Клиент: {CustomerName}, Способ оплаты: {PaymentMethod}, Валюта: {Currency}, " +
            $"Общая сумма: {TotalAmount}");
    }
}


In [ ]:
public class GoodsInvoice : Invoice
{
    public DateTime SupplyDate { get; set; }

    public GoodsInvoice(string invoiceNumber, DateTime issueDate, DateTime supplyDate, string customerName, string paymentMethod, string currency, List<ILineItem> lineItems)
        : base(invoiceNumber, issueDate, customerName, paymentMethod, currency, lineItems)
    {
        SupplyDate = supplyDate;
    }

    public override void PrintInvoice()
    {
        base.PrintInvoice();
        Console.WriteLine($"Дата поставки: {SupplyDate.ToShortDateString()}");
    }
}
ItemCollection<LineItem> goodsCollection = new ItemCollection<LineItem>();
LineItem goodsItem1 = new LineItem("Ноутбук", 1000, 2, 100, "SKU001");
LineItem goodsItem2 = new LineItem("Телефон", 500, 5, 50, "SKU002");

goodsCollection.AddItem(goodsItem1);
goodsCollection.AddItem(goodsItem2);

goodsCollection.CalculateTotal();
goodsCollection.DisplayAllItems();

List<ILineItem> lineItems = new List<ILineItem> { goodsItem1, goodsItem2 };
GoodsInvoice goodsInvoice = new GoodsInvoice("INV001", DateTime.Now, DateTime.Now.AddDays(7), "Иван Иванов", "Кредитная карта", "RUB", lineItems);

goodsInvoice.PrintInvoice();


Общая сумма фактуры: 4350
Товар/Услуга: Ноутбук, Цена: 1000, Количество: 2, Скидка: 100, Общая сумма: 1900
Товар/Услуга: Телефон, Цена: 500, Количество: 5, Скидка: 50, Общая сумма: 2450
Номер фактуры: INV001, Дата выпуска: 07.11.2024, Клиент: Иван Иванов, Способ оплаты: Кредитная карта, Валюта: RUB, Общая сумма: 0
Дата поставки: 14.11.2024


In [ ]:
public class ServiceInvoice : Invoice
{
    public DateTime ServiceDate { get; set; }

    public ServiceInvoice(string invoiceNumber, DateTime issueDate, DateTime serviceDate, string customerName, string paymentMethod, string currency, List<ILineItem> lineItems)
        : base(invoiceNumber, issueDate, customerName, paymentMethod, currency, lineItems)
    {
        ServiceDate = serviceDate;
    }

    public override void AddLine(ILineItem lineItem)
    {
        base.AddLine(lineItem);
        Console.WriteLine($"Добавлена услуга: {lineItem.Description}, Дата услуги: {ServiceDate.ToShortDateString()}");
    }
}
List<ILineItem> serviceItems = new List<ILineItem>
{
    new ServiceLineItem("Консультация", 300, 1, 50, 0, "Онлайн"),
    new ServiceLineItem("Уборка", 200, 2, 30, 0, "Очистка помещений")
};
ServiceInvoice serviceInvoice = new ServiceInvoice("SINV001", DateTime.Now, DateTime.Now.AddDays(3), "Иван Иванов", "Кредитная карта", "RUB", serviceItems);
serviceInvoice.AddLine(serviceItems[0]);
serviceInvoice.AddLine(serviceItems[1]);
serviceInvoice.PrintInvoice();


Общая сумма счета: 1160
Добавлена услуга: Консультация, Дата услуги: 10.11.2024
Общая сумма счета: 1620
Добавлена услуга: Уборка, Дата услуги: 10.11.2024
Номер фактуры: SINV001, Дата выпуска: 07.11.2024, Клиент: Иван Иванов, Способ оплаты: Кредитная карта, Валюта: RUB, Общая сумма: 1620


In [ ]:
public class CombinedInvoice : Invoice
{
    public bool ReturnAllowed { get; set; }

    public CombinedInvoice(string invoiceNumber, DateTime issueDate, bool returnAllowed, string customerName, string paymentMethod, string currency, List<ILineItem> lineItems)
        : base(invoiceNumber, issueDate, customerName, paymentMethod, currency, lineItems)
    {
        ReturnAllowed = returnAllowed;
    }

    public override void CalculateTotal()
    {
        base.CalculateTotal();
        if (ReturnAllowed)
        {
            TotalAmount *= 0.9m;
            Console.WriteLine("Возврат разрешен. Применяется скидка.");
        }
    }
}
List<ILineItem> combinedItems = new List<ILineItem> { goodsItem1, serviceItems[0] };
CombinedInvoice combinedInvoice = new CombinedInvoice("INV003", DateTime.Now, true, "Сидор Сидоров", "Электронный перевод", "USD", combinedItems);
combinedInvoice.PrintInvoice();



Номер фактуры: INV003, Дата выпуска: 07.11.2024, Клиент: Сидор Сидоров, Способ оплаты: Электронный перевод, Валюта: USD, Общая сумма: 0
